In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from  sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_df = pd.read_csv("./datasets/training.csv")
test_df = pd.read_csv("./datasets/test.csv")

In [3]:
train_df = train_df.dropna(axis=0)

In [4]:
X_train_df = train_df["Image"].apply(lambda im: np.fromstring(im, sep=' '))
y_train_df = train_df.drop(["Image"], axis=1)

In [5]:
X_train_df = np.array([i for i in X_train_df])

In [6]:
test_df = test_df["Image"].apply(lambda im: np.fromstring(im, sep=' '))
test_df = np.array([i for i in test_df])

In [62]:
X_train = X_train_df / 255
y_train = y_train_df.values / 96

In [64]:
X_train, X_test ,y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, 
                                                   random_state=42)

In [65]:
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128

# Network Parameters
num_input = 96 * 96 # MNIST data input (img shape: 28*28)
num_classes = 30 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit


In [82]:
def conv_net2(X,n_classes, dropout, reuse, is_training):
    with tf.variable_scope("ConvNet", reuse=reuse):
#         X = tf.cast(X, tf.float32)
        x = tf.reshape(X, shape=[-1, 96, 96, 1])
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        
        fc1 = tf.contrib.layers.flatten(conv1)
        fc1 = tf.layers.dense(fc1, 100)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        out = tf.layers.dense(fc1, 30)
    
    return out

In [83]:
def model_fn(features, labels, mode):
    logits_train = conv_net2(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net2(features, num_classes, dropout, reuse=True, is_training=False)
    
#     pred_classes = tf.argmax(logits_test, axis=1)
    pred_classes = logits_test
#     pred_proba = tf.nn.softmax(logits_test)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
#     loss_op = tf.reduce_mean(tf.pow(logits_train - labels, 2)) / (2 * batch_size)
    loss_op = tf.sqrt( tf.losses.mean_squared_error(logits_train, labels) )
                                                                           
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    estim_specs = tf.estimator.EstimatorSpec(mode=mode,predictions=pred_classes, 
                                        loss=loss_op, train_op=train_op, 
                                        eval_metric_ops={"accuracy":acc_op})
    return estim_specs

In [84]:
model = tf.estimator.Estimator(model_fn)
input_fn = tf.estimator.inputs.numpy_input_fn(x=X_train,
                                              y=y_train, 
                                              batch_size=batch_size, 
                                             num_epochs=None, shuffle=True)
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j0/mlln6qvj2kl42rr7mg80lsxc0000gn/T/tmpwc1vu3b5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a403946d8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/j0

In [85]:
input_fn = tf.estimator.inputs.numpy_input_fn(x=X_test, y=y_test, 
                                              batch_size=batch_size, shuffle=False)
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-30-16:21:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/j0/mlln6qvj2kl42rr7mg80lsxc0000gn/T/tmpwc1vu3b5/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-30-16:21:28
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.0, global_step = 500, loss = 0.056740664


{'accuracy': 0.0, 'global_step': 500, 'loss': 0.056740664}

In [86]:
input_fn = tf.estimator.inputs.numpy_input_fn(x=test_df / 255, shuffle=False)
y_pred = np.array( list(model.predict(input_fn)))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/j0/mlln6qvj2kl42rr7mg80lsxc0000gn/T/tmpwc1vu3b5/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [87]:
y_pred  = y_pred * 96

In [88]:
y_pred = y_pred.clip(0, 96)

In [89]:
idtable = pd.read_csv("./datasets/IdLookupTable.csv")

In [90]:
cols = train_df.columns[:-1]

In [91]:
values = []

In [92]:
for index, row in idtable.iterrows():
     values.append((
        row['RowId'],
        y_pred[row.ImageId - 1][np.where(cols == row.FeatureName)[0][0]],
        ))

In [93]:
submission = pd.DataFrame(values, columns=('RowId', 'Location'))
submission.to_csv('./submission.csv', index=False)